# Cleaning the Tweets for EDA

## Imports

In [238]:
import pandas as pd
from collections import Counter

In [239]:
# READ CSV 
df = pd.read_csv("../datasets/raw_df.csv")

## Clean Data

### Remove Unnecessary Columns

Only the `username` and `text` columns will be kept.

In [240]:
df = df[["username", "text"]]

df.head(5)

,username,text
0,Mult Co Fire/EMS log,"MED - MEDICAL at 700 BLOCK OF NE 78TH AVE, POR..."
1,Mult Co Fire/EMS log,MED - MEDICAL at SE 32ND AVE / SE JOHNSON CREE...
2,MichelleBot,Valley of Fire @ Valley of Fire State Park htt...
3,Mult Co Fire/EMS log,"MED - MEDICAL at 2000 BLOCK OF NE RODNEY AVE, ..."
4,Houdini,Special shout out to @m0llyk4y for comin throu...


### Clean HTML,  Make Lowercase, Remove Links

Many of the texts contain non-letters, uppercase letters, `URL` links, or `hashtags` that will need to be removed for EDA.

**Example Below:**

In [241]:
df["text"][4]

'Special shout out to @m0llyk4y for comin through with the fire video   @ Downtown Long Beach https://www.instagram.com/p/BqiroLnBHjY/?utm_source=ig_twitter_share&igshid=1bftkxvhtu83w\xa0…'

In [242]:
def clean_df(data, col):
    
    # REMOVE URL LINKS 
    data[col] = data[col].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True) 
    # https://stackoverflow.com/questions/51994254/removing-url-from-a-column-in-pandas-dataframe
    
    # REMOVE NON-LETTERS:
    data[col] = data[col].str.replace('[^a-zA-Z]', ' ')
    
    # MAKE LOWERCASE:
    data[col] = data[col].str.lower()
    
    return data

In [243]:
clean_df(df, "text")
clean_df(df, "username")

,username,text
0,mult co fire ems log,med medical at block of ne th ave por...
1,mult co fire ems log,med medical at se nd ave se johnson cree...
2,michellebot,valley of fire valley of fire state park
3,mult co fire ems log,med medical at block of ne rodney ave ...
4,houdini,special shout out to m llyk y for comin throu...
...,...,...
11779,scott sanders,pray for calabasas and thousand oaks this fi...
11780,john meredith,east bay area smoke has turned the sun red ...
11781,wildfire bot wa or unofficial,buttonfire unofficial new fire report detail...
11782,d j grothe,fire haze in sonoma right now too fairmont...


In [244]:
df["text"][4]

'special shout out to  m llyk y for comin through with the fire video     downtown long beach   '

In [245]:
df.to_csv("../datasets/df.csv")

In [95]:
# NEEDS FURTHER IMPLEMENTATION
# df.to_sql("../datasets/twitter_text", con=engine)

## Data for EDA

### Create Keywords

In [246]:
# USING PREBUILT CRISIS LEXICON KEYWORDS
    # https://crisislex.org/crisis-lexicon.html
def create_keywords_list(filepath):
    keywords = open(filepath)
    clean_text = ""
    clean_keywords = []
    keywords_list = []
    result = []
    
    for keyword in keywords:
        keywords_list.append(keyword)

    for words in keywords_list:
        clean_text = words.replace("\n", "")
        clean_keywords.append(clean_text.split())

    for i in clean_keywords:
        for j in i:
            result.append(j)
    return result 

In [247]:
keywords = create_keywords_list("../datasets/CrisisLexLexicon/CrisisLexRec.txt")

In [248]:
def check_keywords(data, col, lst):
    kw_df = pd.DataFrame()
    
    for word in data[col]:
        if word in lst:
            kw_df = kw_df.append(data.loc[data[col].str.contains(word)], ignore_index=True)
    return kw_df

### Explore Usernames

In [261]:
# MAJORITY USERNAMES ARE EMS SERVICES
# THERE ARE BOTS
# THERE ARE EMOJIS

# CREATE A TABLE OF 100 MOST COMMON WORDS IN USERNAMES
Counter(" ".join(df["username"]).split()).most_common(100)
username_most_occur_df = pd.DataFrame(Counter(" ".join(df["username"]).split()).most_common(100),
                          columns=["username", "no_occurences"])

username_most_occur_df

,username,no_occurences
0,fire,4572
1,co,4570
2,log,4569
3,ems,4556
4,mult,4555
...,...,...
95,jon,18
96,portland,18
97,comfort,18
98,ashley,18


In [250]:
username_most_occur_df.to_csv("../datasets/username-most-freq.csv")

In [262]:
username_keywords = check_keywords(username_most_occur_df, "username", keywords)
username_keywords

,username,no_occurences
0,fire,4572
1,wildfire,196
2,city,33
3,news,20


In [216]:
username_keywords.to_csv("../datasets/username-crisisword-match.csv")

### Explore Text

In [252]:
# CREATE A TABLE OF 100 MOST COMMON WORDS IN THE TEXT OF EACH TWEET
text_occur_df = pd.DataFrame(Counter(" ".join(df["text"]).split()).most_common(100),
                             columns=["text", "no_occurences"])
text_occur_df

,text,no_occurences
0,portland,9081
1,fire,8269
2,the,6583
3,of,6328
4,at,5522
...,...,...
95,me,295
96,who,295
97,help,291
98,those,289


In [253]:
text_occur_df.to_csv("../datasets/text-most-freq.csv")

In [254]:
text_keywords = check_keywords(text_occur_df, "text", keywords)
text_keywords

,text,no_occurences
0,fire,8269
1,fires,1188
2,campfire,635
3,medical,3432
4,tonight,629
5,victims,297
6,help,291


In [255]:
text_keywords.to_csv("../datasets/text-crisisword-match.csv")